In [1]:
import numpy as pd
import pandas as pd

from Config import Keys
import Utility.Utility as Utility

from Connector.Client import OrderSide, OrderType
from Connector.Bitmex import Bitmex
from Database.Database import Database

In [2]:
DATA_DIR_PATH = "Data/"

In [3]:
bitmex_client = Bitmex(public_key=Keys.PUBLIC_KEY, secret_key=Keys.SECRET_KEY)

Successfully connected to the Bitmex API.


In [4]:
database = Database(path=DATA_DIR_PATH)

In [7]:
choice = input("Mode: (collect / backtest / trade): ").lower()
symbol = input("Enter a symbol: ").upper()
if choice == "collect":
    df = database.add_data(client=bitmex_client, start="2021-01-01", symbol=str(symbol))
elif choice == "backtest":
    data = bitmex_client.get_historical_data(symbol=str(symbol))
    df = database.get_data(client_name=bitmex_client.name, symbol=str(symbol))
elif choice == "trade":
    pass

Iteration 0: 1000 candles collected | Total candles 1000
Iteration 1: 1000 candles collected | Total candles 2000
Iteration 2: 1000 candles collected | Total candles 3000
Iteration 3: 1000 candles collected | Total candles 4000
Iteration 4: 1000 candles collected | Total candles 5000
Iteration 5: 1000 candles collected | Total candles 6000
Iteration 6: 1000 candles collected | Total candles 7000
Iteration 7: 1000 candles collected | Total candles 8000
Iteration 8: 1000 candles collected | Total candles 9000
Iteration 9: 1000 candles collected | Total candles 10000
Iteration 10: 1000 candles collected | Total candles 11000
Iteration 11: 1000 candles collected | Total candles 12000
Iteration 12: 1000 candles collected | Total candles 13000
Iteration 13: 1000 candles collected | Total candles 14000
Iteration 14: 1000 candles collected | Total candles 15000
Iteration 15: 1000 candles collected | Total candles 16000
Iteration 16: 1000 candles collected | Total candles 17000
Iteration 17: 10

In [9]:
resampled_df = Utility.resample_dataframe(df, "4h")
resampled_df

,open,high,low,close,volume
date,,,,,
2021-01-01 00:00:00+00:00,738.40,750.30,738.40,747.90,2387
2021-01-01 04:00:00+00:00,747.90,749.35,740.50,740.50,464
2021-01-01 08:00:00+00:00,740.50,752.30,740.00,740.75,224
2021-01-01 12:00:00+00:00,740.75,763.35,722.25,737.70,9975
2021-01-01 16:00:00+00:00,737.70,740.50,719.20,727.55,3525
...,...,...,...,...,...
2021-11-19 20:00:00+00:00,3941.10,4850.00,3941.10,4266.80,91
2021-11-20 00:00:00+00:00,4266.80,4370.75,4266.80,4287.15,172
2021-11-20 04:00:00+00:00,4287.15,4319.35,4226.95,4231.50,108
